# BBOX parsing automation

In [90]:
#hacky but it works
def parse_xml_files(path, name_of_file = 'bbox coordinates.txt'):
    '''
    Function to parse xml files into COCO .json format.
    pass in the path to the directory containing all of the
    .xml files you wish to convert, and the name_of_file that
    you wish to write the final bbox coordinates to (default='bbox coordinates.txt').
    If named file does not exist, one will be created.
    '''
    import os
    import ast
    import xmltodict
    import json
    
    os.chdir(path)
    final_dict = {}
    xml_list = []

    #taking .xml files and loading them to list
    for file in os.listdir():
        if '.xml' in file:
            xml_list.append(file)

    #looping through each item in list and converting to dictionary
    for x in range(len(xml_list)):
        with open(xml_list[x]) as fd:
            doc = xmltodict.parse(fd.read(), process_namespaces=True)

        unparsed_dict = ast.literal_eval(json.dumps(doc))
        unparsed_dict_slice = unparsed_dict['annotation']['object']

        attributelist = []
        namelist = []

        #parsing dictionaries to COCO .json format
        for q in range(len(unparsed_dict['annotation']['object'])):
            
            #if/else statement needed for images with only 1 item in them
            
            #if type==dict, then image has 1 item.
            if type(unparsed_dict_slice) == dict:
                item_names = unparsed_dict_slice['name']
                xmin = int(unparsed_dict_slice['bndbox']['xmin'])
                ymin = int(unparsed_dict_slice['bndbox']['ymin'])
                xmax = int(unparsed_dict_slice['bndbox']['xmax'])
                ymax = int(unparsed_dict_slice['bndbox']['ymax'])
                filename = unparsed_dict['annotation']['filename']
                attributes = [ymin, xmin, ymax, xmax]
                attributelist.append(attributes)
                namelist.append(item_names)
                bbox_dict = {filename:ast.literal_eval(str(tuple([[attributelist] + [namelist]])).replace('(','').replace(',)',''))}
                break
            
            #else, loop through all items.
            else:
                item_names = unparsed_dict_slice[q]['name']
                xmin = int(unparsed_dict_slice[q]['bndbox']['xmin'])
                ymin = int(unparsed_dict_slice[q]['bndbox']['ymin'])
                xmax = int(unparsed_dict_slice[q]['bndbox']['xmax'])
                ymax = int(unparsed_dict_slice[q]['bndbox']['ymax'])
                filename = unparsed_dict['annotation']['filename']

            attributes = [ymin, xmin, ymax, xmax]

            attributelist.append(attributes)
            namelist.append(item_names)

            #creating dictionary of the form { 'image': [[[bbox coords], [bbox coords]], ['item','item']] }
            bbox_dict = {filename:ast.literal_eval(str(tuple([[attributelist] + [namelist]])).replace('(','').replace(',)',''))}

        #adding bbox_dict to one main dictionary
        final_dict.update(bbox_dict)

    #writing single dictionary to file
    file = open(name_of_file, 'w')
    file.write(json.dumps(final_dict))
    file.close()

In [216]:
parse_xml_files('C:/Users/HARENB/Desktop/turn these into xml')

In [6]:
def create_csv(name_of_file='tagnames.csv',header='PicNames,Tags'):
    '''
    Function to parse .xml files into the following .csv format:
    
    filename,tag1 tag2 tag3 tag4
    filename1,tag1 tag2 tag3
    
    name_of_file = name of file to write. Default = tagnames.csv
    header = first line of .csv file, contains column names of items in file
        default = 'PicNames,Tags'
    '''
    import fileinput
    
    #opening file, looking for tags
    ffd = open(name_of_file, 'w')
    for file in os.listdir():
        if '.xml' in file:
            g = open(file).read()
            file = file.replace('.xml','')
            ffd.write('\n' + file + ',')
            if 'basket' in g:
                ffd.write('basket ')
            if 'energytag' in g:
                ffd.write('energytag ')
            if 'handle' in g:
                ffd.write('handle ')
            if 'hose' in g:
                ffd.write('hose ')
            if 'litpack' in g:
                ffd.write('litpack ')
            if 'soap' in g:
                ffd.write('soap ')
    ffd.close()
    
    #stripping space at end of each line
    with fileinput.input(files=name_of_file,inplace=True) as ffd:
        for line in ffd:
            line = line.rstrip('\n');

            if line.endswith(' '):
                line = line[:-1]
            print(line)
    
    #adding column names to first line of file
    with open(name_of_file, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(header.rstrip('\r\n') + content)

In [218]:
create_csv()

In [7]:
def create_val_images(path,size=.2):
    '''
    Function to create list of validation images, and
    split them for training.
    size = percentage of images being split into validation images
    path = location of .jpg images
    Run function, then set val_images = function()
    '''
    import os
    
    os.chdir(path)

    jpglist = []
    for file in os.listdir():
        if '.jpg' in file:
            jpglist.append(file)
    index = int(len(jpglist)*.2)
    print(f'Num. items in validation set: {index}')
    return jpglist[:index]